In [61]:
from langchain_ollama.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.prompts import PromptTemplate
from pprint import pprint
from langchain_core.pydantic_v1 import BaseModel, Field

In [62]:

llm =  ChatOllama(model="phi3:3.8b")
# llm =  ChatOllama(model="cow/gemma2_tools:2b")

In [67]:

_template = """
You are a helpful bot. Your task is to find information from patient data. Extracts the following
properties name, gender, age, weight, height, BMI, chief medical complaint.

"name": Name of patient , 

"gender": Gender of patient 

"age": Age of patient

"weight": weight of patient,

"height": height of patient

"BMI": BMI of patient

"chief_medical_complaint": chief_medical_complaint of patient

Always remeber:
- Give parsed information in JSON Format 
- Value should be wrapped in ""
- Dont assume informations. If only available then prase
- If you couldnt find value then value should be "not_given"

Follow this format:

    "key" : "value"

Remember:
If value not available then return value for that key should be "not_given"

patient_data: {patient_data}
data:
"""
prompt = PromptTemplate.from_template(_template)

In [68]:

class Patient(BaseModel):
    name: str = "Name of patient" 
    gender:str  = "Gender of patient" 
    age:str =  "Age of patient"
    weight: str =  "weight of patient,"
    height:str =  "height of patient"
    BMI :str =  "BMI of patient"
    chief_medical_complaint:str =  "chief_medical_complaint of patient"



# llm_ex =  (prompt | llm.with_structured_output(Patient) )
llm_ex =  (prompt | llm | JsonOutputParser())



In [69]:
data1 = """
Patient 1: John R. Whitaker, a 52-year-old male, stands 5'10" (70 inches) tall and weighs 198
lbs. Mr. Whitaker has a history of hypertension and type 2 diabetes, both diagnosed in his
mid-40s, and recently began experiencing worsening peripheral neuropathy in his lower
extremities. He also reports chronic lower back pain, which he attributes to years of heavy
lifting in his previous occupation as a construction worker. Over the past six months, John
has developed shortness of breath during mild exertion, prompting concerns about potential
early-stage congestive heart failure. Additionally, he struggles with obesity-related sleep
apnea, contributing to fatigue and cognitive fog throughout the day. Despite his conditions,
Mr. Whitaker maintains a generally positive outlook but admits to inconsistent medication
adherence and difficulty following a healthy diet.
"""

data2 = """
Patient 2: For the past six months, Emily J. Rivera has been dealing with persistent chest
tightness and shortness of breath, especially during moderate activity. She works as a
schoolteacher and describes her symptoms as worsening under stress, which she initially
dismissed as anxiety. A thorough evaluation revealed mild asthma, along with borderline high
cholesterol levels. Emily is 41 years old, 5'5" (65 inches) tall, and weighs 172 lbs. She also
complains of intermittent joint stiffness in her hands, particularly in the mornings, which her
physician suspects could be early osteoarthritis. Emily’s sedentary lifestyle and inconsistent
exercise routine have contributed to her struggles with maintaining a healthy weight, though
she remains committed to improving her overall health with proper guidance and treatment.
"""


data3 = """
Patient 3: Karen L. Thompson, a 38-year-old female, is 5'4" (64 inches) tall and weighs 162
lbs. She has a history of irritable bowel syndrome (IBS) and recurrent migraines, both of
which have intensified over the past year. Karen also experiences chronic fatigue and joint
pain, leading her physician to investigate possible early-stage rheumatoid arthritis. She
reports frequent episodes of dizziness and occasional heart palpitations, which have been
attributed to mild anemia and elevated stress levels. Karen’s symptoms are exacerbated by
her demanding job as a paralegal, where long hours and poor posture have contributed to
persistent neck and shoulder tension. Recently, she has begun experiencing intermittent
insomnia, further impacting her energy levels and overall well-being.
"""

In [70]:
llm_ex.invoke(data1)

OutputParserException: Invalid json output: {
    "name": "John R. Whitaker",
    "gender":"male",
    "age":52,
    "weight":198,
    "height":70,
    "BMI":30.4 (calculated value),
    "chief_medical_complaint":["worsening peripheral neuropathy in lower extremities", "chronic lower back pain"]
}
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE

In [59]:
llm_ex.invoke(data2)

{'name': 'Emily J. Rivera',
 'gender': 'female',
 'age': 41,
 'weight': 172,
 'height': '5\'5" (65 inches)',
 'BMI': 'not_given',
 'chief_medical_complaint': 'persistent chest tightness and shortness of breath, especially during moderate activity'}

In [60]:
llm_ex.invoke(data3)

{'name': 'Karen L. Thompson',
 'gender': 'female',
 'age': '38',
 'weight': '162 lbs',
 'height': '5\'4"',
 'BMI': 'not_given',
 'chief_medical_complaint': 'dizziness, occasional heart palpitations, chronic fatigue, joint pain, insomnia'}